In [1]:
import pandas as pd
import re
from unidecode import unidecode

def preprocess_text(text):
    text = text.lower().strip()
    text = unidecode(text)
    text = re.sub('[^a-zA-Záéíóúçãõôê\s,]', '', text)
    text = re.sub(',+', ',', text)
    text = re.sub(' +', ' ', text)
    return text

def extract_ingredients(text):
    pattern = r"\s*,\s*|\s+e\s+(?=[a-zA-Z])"
    return [ingredient.strip() for ingredient in re.split(pattern, text)]

def detect_allergens(ingredients):
    allergens_set = {"leite", "ovo", "amendoim", "soja", "trigo", "centeio", "cevada"}
    return [ingredient for ingredient in ingredients if any(allergen in ingredient for allergen in allergens_set)]

df = pd.read_csv('openfoodfacts_export.csv', sep="\t", low_memory=False, usecols=["product_name_pt","ingredients_text_pt"])

df.dropna(subset=['ingredients_text_pt'], inplace=True)

df['ingredients_text_pt'] = df['ingredients_text_pt'].apply(preprocess_text)

df['alergenos'] = df['ingredients_text_pt'].apply(lambda x: detect_allergens(extract_ingredients(x)))

display(df)


,product_name_pt,ingredients_text_pt,alergenos
1,Thé vert au jasmin,milho,[]
3,Pão para hamburger tradicional,farinha de trigo enriquecida com ferro e acido...,"[farinha de trigo enriquecida com ferro, glute..."
8,NaN,"batata, oleo misto vegetal de palma e soja e s...","[soja, sal alergicos contem derivados de soja,..."
26,Wafer recheado sabor Chocolate e Avelã,"acucar, gordura vegetal, farinha de trigo enri...","[farinha de trigo enriquecida com ferro, leite..."
27,Feijão preto,feijao preto,[]
...,...,...,...
5078,NaN,informacao nutricional informacion nutricional...,[]
5079,NaN,oleos vegetals quidos e interesterificados con...,[lecitina de soja]
5089,Aveia em flocos finos,graos laminados de aveia em particulas finas\n...,"[suas estirpes hibridizadas pode conter trigo,..."
5095,Manteiga com sal,"creme de leite pasteurizado, cloreto de sodio ...",[creme de leite pasteurizado]
